<a href="https://colab.research.google.com/github/FerreiraBX95/Master-Thesis---Bruno-Xavier-Ferreira/blob/Development-of-models-for-measurement-of-pH-in-atmospheric-and-pressurized-systems-using-artificial-intelligent-strategies/Code_for_training_the_prediction_CNN_models_Input_4%2C_5_and_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*** Code for training the prediction CNN models - Input 4, 5 and 6***

Inicialização do Weights and Bias (WandB)

In [ ]:
# WandB – Install the W&B library
%pip install wandb -q
#!pip install wandb --upgrade
import wandb
from wandb.keras import WandbCallback

Informações da GPU a ser utilizada

In [ ]:
# Para usar a GPU eh preciso antes "Alterar o tipo de ambiene de Execução" para GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print('Informacao da GPU: ')
  print(gpu_info)

Atualizações de programas (se necessario)

In [ ]:
%pip install scikit-plot -q

In [ ]:
%pip install scikit-learn -q

In [ ]:
#!pip install keras --upgrade
#!pip install tensorflow --upgrade
#!pip install sklearn --upgrade


In [ ]:
%pip install keras-metrics -q

Bibliotecas

In [ ]:
# Bibliotecas

import os
import random

import cv2
from matplotlib import image
from matplotlib import pyplot

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout

from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


from tensorflow.keras import metrics


from sklearn.metrics import roc_auc_score 
from sklearn.metrics import f1_score 
from sklearn.metrics import fbeta_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score 

from skimage.transform import resize
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_confusion_matrix

import warnings
warnings.filterwarnings('ignore')

from skimage import data
from skimage.color import rgb2hsv

Carregamento e pre-processamento dos dados

In [ ]:
## Teste para leitura de imagens do drive
address = '/content/drive/MyDrive/Mestrado_Bruno Xavier/Dissertacao de Mestrado - Cod/Banco_de_Imagens_Curva_de_calib_PYTHON_med_pH_Alline_ajust'
CATEGORIES = ["pH_2","pH_3","pH_4","pH_5","pH_6","pH_7","pH_8","pH_9","pH_10"]

tam = 313

#x_db = np.zeros((tam,800,1280),np.int16)
x_db = np.zeros((tam,280,280),np.int16)

ind = 0
for category in CATEGORIES:  
    path = os.path.join(address,category)  # create path 
    for img in os.listdir(path):  # iterate over each image 
        img_cap = image.imread(os.path.join(path,img))  # convert to array
        img_cap = img_cap[300:580,450:730,:]
        img_cap = rgb2hsv(img_cap)
        #img_cap = img_cap[:, :, 0]    #hue
        #img_cap = img_cap[:, :, 1]    #sat
        img_cap = img_cap[:, :, 2]    #value
        if category == 'pH_2':
          x_db[ind,:,:] = img_cap
        if category == 'pH_3':
          x_db[ind,:,:] = img_cap
        if category == 'pH_4':
          x_db[ind,:,:] = img_cap
        if category == 'pH_5':
          x_db[ind,:,:] = img_cap
        if category == 'pH_6':
          x_db[ind,:,:] = img_cap
        if category == 'pH_7':
          x_db[ind,:,:] = img_cap
        if category == 'pH_8':
          x_db[ind,:,:] = img_cap
        if category == 'pH_9':
          x_db[ind,:,:] = img_cap
        if category == 'pH_10':
          x_db[ind,:,:] = img_cap
        
        ind = ind + 1


In [ ]:
address3 = '/content/drive/MyDrive/Mestrado_Bruno Xavier/Dissertacao de Mestrado - Cod/Valor_pH-DB_med_pH_Alline_ajust.txt'
arq = open(address3,'r')
#y_db = np.zeros((tam,1),np.int16)
y_db = np.zeros((tam,1),np.float)
for i in range(tam):
  linha = arq.readline()
  val = linha.split()
  y_db[i,:] = val

In [ ]:
print('Tam vetor x: ',len(x_db))
print('Tam vetor y: ',len(y_db))


In [ ]:
# Pode ser preciso rodar duas vezez
x_train, x_test, y_train, y_test = train_test_split(x_db, y_db, test_size=0.3,random_state=1)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5,random_state=1)

In [ ]:
print('Tam vetor y_train: ',len(y_train))
#print(y_train)
print('Tam vetor y_train: ',len(y_test))
#print(y_test)
print('Tam vetor y_val: ',len(y_val))
#print(y_val)
print('Tam vetor y: ',len(y_db))
print('Tam vetor y_som: ',len(y_train)+len(y_test)+len(y_val))

Função para a criacao do modelo CNN

In [ ]:
def Model(filt_siz_1,filt_siz_2,ker_siz_1,ker_siz_2,
          n_layers,p_dropout,siz_dense_1,siz_dense_2,act_func_1,act_func_2):

  #inputs = keras.layers.Input(shape=(800, 1280, 1))
  inputs = keras.layers.Input(shape=(280, 280, 1))

  if n_layers==2:
    x = keras.layers.Conv2D(filters=filt_siz_1, kernel_size=(ker_siz_1,ker_siz_1), activation=act_func_1)(inputs)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)
    x = keras.layers.Conv2D(filters=filt_siz_2, kernel_size=(ker_siz_2,ker_siz_2), activation=act_func_2)(x)
  if n_layers==3:
    x = keras.layers.Conv2D(filters=filt_siz_1, kernel_size=(ker_siz_1,ker_siz_1), activation=act_func_1)(inputs)
    x = keras.layers.Conv2D(filters=filt_siz_1, kernel_size=(ker_siz_1,ker_siz_1), activation=act_func_1)(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)
    x = keras.layers.Conv2D(filters=filt_siz_2, kernel_size=(ker_siz_2,ker_siz_2), activation=act_func_2)(x)
  if n_layers==4:
    x = keras.layers.Conv2D(filters=filt_siz_1, kernel_size=(ker_siz_1,ker_siz_1), activation=act_func_1)(inputs)
    x = keras.layers.Conv2D(filters=filt_siz_1, kernel_size=(ker_siz_1,ker_siz_1), activation=act_func_1)(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)
    x = keras.layers.Conv2D(filters=filt_siz_2, kernel_size=(ker_siz_2,ker_siz_2), activation=act_func_2)(x)
    x = keras.layers.Conv2D(filters=filt_siz_2, kernel_size=(ker_siz_2,ker_siz_2), activation=act_func_2)(x)

  x = keras.layers.GlobalAveragePooling2D()(x)

  x = keras.layers.Dense(siz_dense_1, activation='relu')(x)
  x = keras.layers.Dropout(p_dropout)(x)
  x = keras.layers.Dense(siz_dense_2, activation='relu')(x)
  
  outputs = keras.layers.Dense(1)(x)
  #outputs = keras.layers.Dense(len(CATEGORIES), activation='softmax')(x)

  return keras.models.Model(inputs=inputs, outputs=outputs)

Treinamento do modelo com sweep wandb

In [ ]:
#project_name="sweeps-CNN_regre_pH_teste_9-med-pH-Alline-ajust-crop-hsv_hue"
#project_name="sweeps-CNN_regre_pH_teste_9-med-pH-Alline-ajust-crop-hsv_sat"
project_name="sweeps-CNN_regre_pH_teste_9-med-pH-Alline-ajust-crop-hsv_value"

In [ ]:
def train():
    # Specify the hyperparameter to be tuned along with
    # an initial value
    config_defaults = {
        'epochs': 60,
        'batch_size': 8,
        'n_layers': 2,
        'activation_1': 'relu',
        'activation_2': 'relu',
        'filters_size_1':8,
        'filters_size_2':8,
        'kernel_size_1': 3,
        'kernel_size_2': 3,
        'p_dropout':0.2,
        'learning_rate': 0.001,
        'siz_dense_1':128,
        'siz_dense_2':32
    }

    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)

    ## Prepare trainloader
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    trainloader = trainloader.shuffle(1024).batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    # prepare valloader 
    valloader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    valloader = valloader.batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    # prepare testloader 
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    testloader = testloader.batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    # Iniialize model with hyperparameters
    keras.backend.clear_session()
    model = Model(wandb.config.filters_size_1,wandb.config.filters_size_2, wandb.config.kernel_size_1,wandb.config.kernel_size_2,
                  wandb.config.n_layers,wandb.config.p_dropout,wandb.config.siz_dense_1,wandb.config.siz_dense_2,
                  wandb.config.activation_1,wandb.config.activation_2)
    
    model.summary()
    # Compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=wandb.config.learning_rate) # optimizer with different learning rate specified by config
    model.compile(opt, loss='mean_squared_error', metrics=[MeanAbsoluteError(),RootMeanSquaredError()])

    # Train the model
    treinamento = model.fit(trainloader, epochs=wandb.config.epochs, validation_data= valloader, callbacks=[WandbCallback()]) # WandbCallback to automatically track metrics
    
    #Performance
    mse_train, mae_train, rmse_train = model.evaluate(trainloader)
    mse_val, mae_val, rmse_val = model.evaluate(valloader)
    mse_test, mae_test, rmse_test = model.evaluate(testloader)
 
    wandb.log({"MAE_train": mae_train})
    wandb.log({"MSE_train": mse_train})
    wandb.log({"RMSE_val": rmse_train})
    wandb.log({"MAE_val": mae_val})
    wandb.log({"MSE_val": mse_val})
    wandb.log({"RMSE_val": rmse_val})
    wandb.log({"MAE_test": mae_test})
    wandb.log({"MSE_test": mse_test})
    wandb.log({"RMSE_test": rmse_test})
    ##  TesteGeral
    db_loader = tf.data.Dataset.from_tensor_slices((x_db, y_db))
    db_loader = db_loader.shuffle(1024).batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    mse_db, mae_db, rmse_db = model.evaluate(db_loader)
    wandb.log({"MAE_geral": mae_db})
    wandb.log({"MSE_geral": mse_db})
    wandb.log({"RMSE_geral": rmse_db})   
    


Sweep config (Definição dos parametros a serem otimizados e suas faixas)

In [ ]:
sweep_config = {
  'method': 'bayes', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 50
  },
  'parameters': {
        'epochs': {
            'distribution': 'int_uniform',
            'max': 120,
            'min': 80
        },
        'batch_size': {
            'values': [4,8,12]
        },
        'n_layers': {
            'values': [2, 3, 4]
        },
        'activation_1': {
            'distribution': 'categorical',
            'values':['linear', 'relu', 'sigmoid','tanh']
        },
        'activation_2': {
            'distribution': 'categorical',
            'values':['linear', 'relu', 'sigmoid','tanh']
        },
        'filters_size_1': {
            'values': [4, 8]
        }, 
        'filters_size_2': {
            'values': [4, 8]
        },
        'kernel_size_1': {
            'values': [1,3,5]
        },
        'kernel_size_2': {
            'values': [1,3,5]
        },
        'p_dropout': {
            'values': [0.05,0.1,0.15,0.2,0.25,0.3]
        },
        'learning_rate':{
            'values': [0.01, 0.005, 0.001, 0.0005, 0.0001]
        },
        'siz_dense_1': {
            'values': [40,50,60,70,80,90,100,120]
        },
        'siz_dense_2': {
            'values': [10,20,30,40,50,60]
        }
  }
}

Inicializar um novo projeto no sweep config

In [ ]:
sweep_id = wandb.sweep(sweep_config, project=project_name)

Ativando o sweep

In [ ]:
wandb.agent(sweep_id, function=train)